# Numpy Memory Array for Fast Image Loading

In [ ]:
%load_ext autoreload
%autoreload 2

In [98]:
from pathlib import Path

import cv2
import numpy as np
from PIL import Image

from model.memory_map import MemMapReader, MemMapWriter

## Writer

In [99]:
image_paths = list(Path('outputs/speed_test').glob('*.png'))
mm_writer = MemMapWriter(image_paths, 'outputs/mm/mem_map.dat')
mm_writer.write()
print(mm_writer)

Saving memmap:   0%|          | 0/476 [00:00<?, ?it/s]

Saving memmap: 100%|██████████| 476/476 [00:00<00:00, 3107.33it/s]

MemMap with 476 images:
- Output file: outputs/mm/mem_map.dat
- Resize: (256, 256)
- dtype: uint8
- func: crop_resize_driver


## Reader

In [100]:
mm_reader = MemMapReader('outputs/mm/mem_map.dat', (256, 256))

In [101]:
len(mm_reader)

476

In [102]:
mm_reader[len(mm_reader) - 1].shape

(256, 256)

### Memory Array

Memory array creates **read-only** numpy array.

In [103]:
%%timeit -n 10

for img in mm_reader:
    x = img.shape

316 μs ± 36.4 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [104]:
%%timeit -n 10

for img in mm_reader:
    x = np.copy(img).shape

1.82 ms ± 96.1 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [105]:
%%timeit -n 10

for imgs in mm_reader.iter_windows(4):
    x = (len(imgs), imgs[0].shape)

1.45 ms ± 139 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [106]:
%%timeit -n 10

for imgs in mm_reader.iter_windows_mut(4):
    x = (len(imgs), imgs[0].shape)

5.92 ms ± 490 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Pillow

In [107]:
%%timeit -n 10

for img_path in image_paths:
    x = np.array(Image.open(img_path)).shape

112 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [108]:
%%timeit -n 10

for i in range(0, len(image_paths), 4):
    xs = [np.array(Image.open(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

110 ms ± 2.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### OpenCV

In [109]:
%%timeit -n 10

for img_path in image_paths:
    x = cv2.imread(str(img_path)).shape

106 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [110]:
%%timeit -n 10

for i in range(0, len(image_paths), 4):
    xs = [cv2.imread(str(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

110 ms ± 2.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Benchmark Result

We used [`timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) magic command to measure the time.

Memory array is ~20 times faster than Pillow or OpenCV.